<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-07-27 10:11:04


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
def print_distribution():

  # star stocks
  print('Overall - Star Stocks:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('Overall - V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Overall - Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Overall - Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

  # stocks in portfolio
  print('Portfolio - Stocks:')
  print(df_mypf['InPortfolio'].value_counts())
  print('-------')

  # star stocks in portfolio
  print('Portfolio - Star Stocks:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())
  print('-------')

  # latest quarter in portfolio
  print('Portfolio - Latest Quarter:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['LatestQtr'].value_counts())
  print('-------')

  # criteria of portfolio stocks
  print('Portfolio - Criteria:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())
  print('-------')

  # strategies in portfolio stocks
  print('Portfolio - Strategy:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())
  print('-------')

  # conviction distribution
  print('Portfolio - Conviction:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Overall - Star Stocks:
1.0    93
0.0    92
Name: StarStock, dtype: int64
-------
Overall - V40/V40N Stocks:
V40N    39
V40     38
VR      23
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
-------
Overall - Latest Qtr:
0    141
1     46
Name: LatestQtr, dtype: int64
-------
Overall - Star + V40/V40N Stocks:
1.0    42
0.0    35
Name: StarStock, dtype: int64
-------
Portfolio - Stocks:
0    135
1     52
Name: InPortfolio, dtype: int64
-------
Portfolio - Star Stocks:
1.0    34
0.0    18
Name: StarStock, dtype: int64
-------
Portfolio - Latest Quarter:
0    36
1    16
Name: LatestQtr, dtype: int64
-------
Portfolio - Criteria:
VR      23
V40N    11
V40     11
AR       4
V200     3
Name: Criteria, dtype: int64
-------
Portfolio - Strategy:
ATH     36
2T3Y    14
BTT      2
Name: Strategy, dtype: int64
-------
Portfolio - Conviction:
M    27
L    14
H    11
Name: Conviction, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
df_common_features

,Date,Close,200_SMA,Dev%_200,Stock,Max,RSI_14,Prev_Close,Avg_Cost,Shares,Med_PE,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction
1231,2023-07-27 00:00:00+05:30,27932.0,23626.0,18.23,3MINDIA.NS,29165.0,57.0,27956.0,NaN,NaN,NaN,69.9,31.40,23.40,V40N,NaN,NaN,0,1.0,M
1232,2023-07-27 00:00:00+05:30,463.0,324.0,42.92,5PAISA.NS,553.0,75.0,464.0,347.06,228.0,NaN,28.0,12.40,10.60,V40N,ATH,532.0,1,1.0,M
1186,2023-07-27 00:00:00+05:30,1617.0,1738.0,-6.99,AAVAS.NS,3196.0,70.0,1616.0,1453.92,37.0,NaN,29.8,9.50,14.10,VR,2T3Y,3075.0,0,1.0,M
1231,2023-07-27 00:00:00+05:30,23185.0,20990.0,10.46,ABBOTINDIA.NS,23533.0,56.0,22945.0,NaN,NaN,NaN,51.9,41.10,31.60,V40,BTT,27500.0,0,1.0,M
1232,2023-07-27 00:00:00+05:30,1904.0,2023.0,-5.87,ACC.NS,2720.0,66.0,1922.0,NaN,NaN,NaN,35.8,10.00,7.06,NaN,NaN,NaN,0,0.0,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2023-07-27 00:00:00+05:30,766.0,817.0,-6.29,VOLTAS.NS,1316.0,47.0,765.0,821.00,NaN,43.4,105.0,9.74,4.42,NaN,NaN,NaN,0,0.0,L
1232,2023-07-27 00:00:00+05:30,1420.0,1428.0,-0.56,WHIRLPOOL.NS,2667.0,39.0,1422.0,NaN,NaN,72.2,82.3,8.87,6.37,V40,NaN,NaN,0,0.0,M
1232,2023-07-27 00:00:00+05:30,405.0,391.0,3.50,WIPRO.NS,713.0,55.0,403.0,388.07,195.0,20.9,19.1,17.70,15.90,VR,2T3Y,666.0,1,1.0,L
1232,2023-07-27 00:00:00+05:30,17.0,17.0,0.59,YESBANK.NS,383.0,51.0,17.0,NaN,NaN,NaN,64.2,4.94,1.99,NaN,NaN,NaN,1,0.0,L


In [11]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-06-28 00:00:00+05:30,18972.0,17947.0,5.71,^NSEI,19979.0,66.0
2023-07-06 00:00:00+05:30,19497.0,17983.0,8.42,^NSEI,19979.0,78.0
2023-07-13 00:00:00+05:30,19414.0,18025.0,7.70,^NSEI,19979.0,69.0
2023-07-20 00:00:00+05:30,19979.0,18090.0,10.44,^NSEI,19979.0,81.0
2023-07-27 00:00:00+05:30,19811.0,18156.0,9.11,^NSEI,19979.0,68.0


In [12]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

#df_common_features['Stock'] = df_common_features['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [13]:
df_common_features['Discount%PE'] = round((df_common_features['Med_PE']-df_common_features['Curr_PE'])*100/(df_common_features['Med_PE']),0)
df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
df_common_features['Upside%Target'] = round((df_common_features['Target']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features = df_common_features.set_index('Stock')

In [14]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Upside%Target',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'RSI_14', 'Curr_ROCE%', 'Curr_ROE%', 'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_common_features[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 52


,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1222.0,-13.00,219.0,1225.87,45.2,24.70,67.0,14.80,10.90,L,45.0,0,0.0,VR,2T3Y
VALIANTORG.NS,507.0,-6.86,206.0,532.61,24.0,16.20,41.0,14.80,13.00,L,33.0,0,0.0,VR,2T3Y
BANDHANBNK.NS,222.0,-5.66,170.0,244.52,NaN,17.60,44.0,6.67,11.90,M,NaN,1,0.0,VR,2T3Y
NIACL.NS,124.0,12.27,169.0,117.07,NaN,19.50,63.0,4.86,4.06,L,NaN,0,0.0,VR,ATH
VAIBHAVGBL.NS,344.0,8.96,154.0,317.48,35.4,53.70,69.0,11.10,9.02,L,-52.0,0,0.0,VR,2T3Y
LUXIND.NS,1681.0,11.64,150.0,1493.72,21.4,36.30,78.0,12.70,9.95,M,-70.0,0,0.0,V40N,2T3Y
PNB.NS,63.0,26.50,138.0,54.97,NaN,15.80,66.0,4.11,3.34,M,NaN,0,1.0,VR,ATH
METROPOLIS.NS,1378.0,0.63,125.0,1365.98,NaN,49.50,42.0,17.40,15.20,L,NaN,0,0.0,VR,ATH
JMFINANCIL.NS,75.0,9.93,113.0,69.17,NaN,12.00,58.0,9.42,7.55,L,NaN,0,0.0,VR,2T3Y


In [15]:
# top from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
CLEAN.NS,1300.0,-10.16,92.0,1365.11,NaN,46.8,33.0,44.50,33.2,L,NaN,0,1.0,VR,2T3Y
FINEORG.NS,4520.0,-12.37,55.0,4697.43,NaN,24.8,33.0,65.40,49.4,M,NaN,0,1.0,V40N,ATH
SRF.NS,2149.0,-8.70,24.0,2222.00,NaN,33.3,35.0,22.40,22.8,M,NaN,1,0.0,VR,ATH
RAJESHEXPO.NS,512.0,-22.78,63.0,578.92,NaN,10.6,38.0,10.30,10.5,M,NaN,0,1.0,V40N,ATH
TEAMLEASE.NS,2325.0,-4.88,96.0,2279.93,NaN,35.4,41.0,14.80,15.1,M,NaN,0,1.0,V40N,2T3Y
VALIANTORG.NS,507.0,-6.86,206.0,532.61,24.0,16.2,41.0,14.80,13.0,L,33.0,0,0.0,VR,2T3Y
IRCTC.NS,621.0,-4.30,45.0,633.05,77.4,50.4,43.0,59.00,45.2,M,35.0,0,1.0,AR,ATH
JUBLFOOD.NS,470.0,-4.94,70.0,471.56,90.5,115.0,43.0,16.00,17.7,M,-27.0,1,0.0,AR,ATH
BANDHANBNK.NS,222.0,-5.66,170.0,244.52,NaN,17.6,44.0,6.67,11.9,M,NaN,1,0.0,VR,2T3Y


In [16]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Upside%Target,Avg_Cost,Med_PE,Curr_PE,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PIDILITIND.NS,2611.0,3.89,6.0,2504.06,94.6,104.0,48.0,24.6,18.7,H,-10.0,0,1.0,V40,ATH
BAJAJFINSV.NS,1651.0,11.27,9.0,1454.93,39.2,41.0,67.0,12.7,14.8,H,-5.0,0,1.0,V40,ATH
EICHERMOT.NS,3361.0,0.38,10.0,3260.20,44.7,31.5,48.0,27.4,21.1,H,30.0,0,1.0,V40N,ATH
5PAISA.NS,463.0,42.92,15.0,347.06,NaN,28.0,75.0,12.4,10.6,M,NaN,1,1.0,V40N,ATH
TCS.NS,3391.0,4.73,15.0,3231.00,28.1,28.4,56.0,59.1,46.9,H,-1.0,1,1.0,V40,ATH
ICICIGI.NS,1371.0,17.11,17.0,1177.58,46.2,38.0,61.0,21.2,17.6,H,18.0,1,1.0,V40,ATH
BATAINDIA.NS,1704.0,6.96,17.0,1523.62,126.0,68.0,63.0,19.6,19.8,H,46.0,0,0.0,V40,ATH


In [17]:
df_common_features.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_common_features[~df_common_features.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]

In [18]:
# star + v40/v40n stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=False)

qualified stocks: 5


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
ASTRAZEN.NS,3834.0,3366.0,13.90,4631.0,64.0,3804.0,30.9,23.0,V40N,NaN,4631.0,0,1.0,M,21.0
BAJFINANCE.NS,7479.0,6552.0,14.15,7860.0,54.0,7433.0,11.8,23.5,V40,BTT,9150.0,0,1.0,H,22.0
BERGEPAINT.NS,675.0,608.0,10.96,858.0,52.0,674.0,23.8,20.4,V40,NaN,858.0,0,1.0,M,27.0
ASIANPAINT.NS,3389.0,3035.0,11.67,3544.0,51.0,3373.0,35.2,27.7,V40,BTT,4250.0,1,1.0,H,25.0
DIXON.NS,4104.0,3655.0,12.28,5750.0,44.0,4087.0,25.8,22.4,V40N,NaN,5750.0,1,1.0,M,40.0


In [19]:
# star + v40/v40n stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 9


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
3MINDIA.NS,27932.0,23626.0,18.23,29165.0,57.0,27956.0,31.40,23.4,V40N,NaN,29165.0,0,1.0,M,4.0
CERA.NS,7740.0,6229.0,24.26,8036.0,56.0,7792.0,25.80,19.4,V40N,NaN,8036.0,0,1.0,H,4.0
ITC.NS,472.0,383.0,23.32,492.0,55.0,472.0,39.20,29.1,V40,NaN,492.0,0,1.0,H,4.0
TITAN.NS,3000.0,2632.0,13.98,3134.0,52.0,3008.0,25.10,30.8,V40,NaN,3134.0,0,1.0,H,4.0
BAJAJ-AUTO.NS,4883.0,3841.0,27.14,4918.0,64.0,4870.0,26.20,20.2,V40,NaN,4918.0,1,1.0,L,1.0
AKZOINDIA.NS,2740.0,2275.0,20.44,2740.0,71.0,2673.0,34.70,26.1,V40,BTT,2740.0,0,1.0,M,0.0
EQUITASBNK.NS,101.0,67.0,50.22,101.0,77.0,96.0,9.24,13.5,V40N,NaN,101.0,0,1.0,M,0.0
ICICIBANK.NS,1001.0,904.0,10.70,1001.0,75.0,996.0,6.82,18.7,V40,NaN,1001.0,1,1.0,H,0.0
POLYCAB.NS,4716.0,3053.0,54.47,4717.0,82.0,4674.0,27.10,20.0,V40N,NaN,4717.0,1,1.0,M,0.0


In [20]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 7


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
UPL.NS,631.0,714.0,-11.65,832.0,34.0,632.0,15.80,13.50,NaN,NaN,832.0,0,1.0,L,32.0
GODREJIND.NS,478.0,447.0,7.02,644.0,44.0,476.0,9.75,11.20,NaN,NaN,644.0,0,1.0,L,35.0
TATAPOWER.NS,221.0,211.0,4.76,285.0,52.0,222.0,12.40,12.60,NaN,NaN,285.0,0,1.0,M,29.0
ADANITRANS.NS,808.0,1683.0,-51.99,4105.0,56.0,815.0,10.50,11.60,NaN,NaN,4105.0,0,1.0,L,408.0
SBICARD.NS,875.0,807.0,8.48,1133.0,58.0,871.0,13.00,25.70,NaN,NaN,1133.0,0,1.0,M,29.0
ADANIENT.NS,2454.0,2702.0,-9.19,4161.0,59.0,2471.0,10.10,9.63,NaN,NaN,4161.0,0,1.0,M,70.0
ADANIGREEN.NS,1134.0,1331.0,-14.82,2970.0,81.0,1140.0,7.81,22.80,NaN,NaN,2970.0,0,1.0,L,162.0


In [21]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
print(tmp_df.index)
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 9
Index(['3MINDIA.NS', 'AKZOINDIA.NS', 'BAJAJ-AUTO.NS', 'CERA.NS',
       'EQUITASBNK.NS', 'ICICIBANK.NS', 'ITC.NS', 'POLYCAB.NS', 'TITAN.NS'],
      dtype='object', name='Stock')


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
AKZOINDIA.NS,2740.0,2275.0,20.44,2740.0,71.0,2673.0,34.70,26.1,V40,BTT,2740.0,0,1.0,M,0.0
EQUITASBNK.NS,101.0,67.0,50.22,101.0,77.0,96.0,9.24,13.5,V40N,NaN,101.0,0,1.0,M,0.0
ICICIBANK.NS,1001.0,904.0,10.70,1001.0,75.0,996.0,6.82,18.7,V40,NaN,1001.0,1,1.0,H,0.0
POLYCAB.NS,4716.0,3053.0,54.47,4717.0,82.0,4674.0,27.10,20.0,V40N,NaN,4717.0,1,1.0,M,0.0
BAJAJ-AUTO.NS,4883.0,3841.0,27.14,4918.0,64.0,4870.0,26.20,20.2,V40,NaN,4918.0,1,1.0,L,1.0
3MINDIA.NS,27932.0,23626.0,18.23,29165.0,57.0,27956.0,31.40,23.4,V40N,NaN,29165.0,0,1.0,M,4.0
CERA.NS,7740.0,6229.0,24.26,8036.0,56.0,7792.0,25.80,19.4,V40N,NaN,8036.0,0,1.0,H,4.0
ITC.NS,472.0,383.0,23.32,492.0,55.0,472.0,39.20,29.1,V40,NaN,492.0,0,1.0,H,4.0
TITAN.NS,3000.0,2632.0,13.98,3134.0,52.0,3008.0,25.10,30.8,V40,NaN,3134.0,0,1.0,H,4.0


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 2


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
NESTLEIND.NS,22740.0,20370.0,11.63,23138.0,50.0,22800.0,138.0,108.0,V40,NaN,23138.0,0,0.0,H,2.0
PGHH.NS,15792.0,13985.0,12.92,15986.0,66.0,15556.0,110.0,79.5,V40,NaN,15986.0,0,0.0,L,1.0


In [23]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 0


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,


In [24]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 8


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,970.0,1127.0,-13.93,3219.0,29.0,1049.0,1000.00,1000.00,V40N,NaN,3219.0,0,0.0,M,232.0
TATAELXSI.NS,7244.0,6808.0,6.40,10602.0,37.0,7227.0,60.00,39.30,V40N,NaN,10602.0,0,0.0,M,46.0
WHIRLPOOL.NS,1420.0,1428.0,-0.56,2667.0,39.0,1422.0,8.87,6.37,V40,NaN,2667.0,0,0.0,M,88.0
SYMPHONY.NS,876.0,941.0,-6.86,1468.0,46.0,875.0,14.80,13.50,V40N,NaN,1468.0,0,0.0,M,68.0
SFL.NS,1180.0,1193.0,-1.09,1972.0,51.0,1175.0,13.60,12.50,V40N,NaN,1972.0,0,0.0,M,67.0
PFIZER.NS,3863.0,4013.0,-3.75,6009.0,56.0,3850.0,27.20,19.80,V40,NaN,6009.0,0,0.0,L,56.0
TTKPRESTIG.NS,780.0,780.0,0.03,1156.0,63.0,779.0,18.20,13.80,V40N,NaN,1156.0,0,0.0,M,48.0
LALPATHLAB.NS,2442.0,2157.0,13.22,4186.0,68.0,2417.0,18.50,15.00,V40N,NaN,4186.0,0,0.0,M,71.0


In [25]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI_14', ascending=True)

qualified stocks: 8


,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,970.0,1127.0,-13.93,3219.0,29.0,1049.0,1000.00,1000.00,V40N,NaN,3219.0,0,0.0,M,232.0
TATAELXSI.NS,7244.0,6808.0,6.40,10602.0,37.0,7227.0,60.00,39.30,V40N,NaN,10602.0,0,0.0,M,46.0
WHIRLPOOL.NS,1420.0,1428.0,-0.56,2667.0,39.0,1422.0,8.87,6.37,V40,NaN,2667.0,0,0.0,M,88.0
SYMPHONY.NS,876.0,941.0,-6.86,1468.0,46.0,875.0,14.80,13.50,V40N,NaN,1468.0,0,0.0,M,68.0
SFL.NS,1180.0,1193.0,-1.09,1972.0,51.0,1175.0,13.60,12.50,V40N,NaN,1972.0,0,0.0,M,67.0
PFIZER.NS,3863.0,4013.0,-3.75,6009.0,56.0,3850.0,27.20,19.80,V40,NaN,6009.0,0,0.0,L,56.0
TTKPRESTIG.NS,780.0,780.0,0.03,1156.0,63.0,779.0,18.20,13.80,V40N,NaN,1156.0,0,0.0,M,48.0
LALPATHLAB.NS,2442.0,2157.0,13.22,4186.0,68.0,2417.0,18.50,15.00,V40N,NaN,4186.0,0,0.0,M,71.0


In [26]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_common_features[df_common_features.index == stock_id]

,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Upside%Target
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1527.0,1595.0,-4.25,1917.0,68.0,1530.0,1000.0,1000.0,NaN,NaN,1917.0,0,0.0,L,26.0


In [27]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [28]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-06-28 00:00:00+05:30,679.0,605.0,12.16,BERGEPAINT.NS,858.0,67.0
2023-07-06 00:00:00+05:30,681.0,605.0,12.50,BERGEPAINT.NS,858.0,65.0
2023-07-13 00:00:00+05:30,662.0,606.0,9.16,BERGEPAINT.NS,858.0,46.0
2023-07-20 00:00:00+05:30,684.0,607.0,12.77,BERGEPAINT.NS,858.0,63.0
2023-07-27 00:00:00+05:30,674.0,608.0,10.94,BERGEPAINT.NS,858.0,52.0
